# Molecular Dynamics: Lennard-Jones system

This notebook performs the same simulation as the one we just examined; however, here the number of cells has been reduced so there is a lower chance of Jupyter having issues because cells have been executed in the wrong order!

In [ ]:
%matplotlib inline

import mbuild as mb
import hoomd
import hoomd.md
import hoomd.deprecated
from matplotlib import pyplot as plt
import random
import nglview as nv
import mdtraj as md
import numpy as np

sigma = 1.0
epsilon = 1.0

# visualize the potential energy function
r = np.linspace(start=.9, stop=sigma+2, num=50,)
potential = (4 * epsilon) * ( (sigma / r)**12 - (sigma/r)**6 )
plt.plot(r[:], potential[:])

In [ ]:
# building the system in mbuild
lj = mb.Compound(name='C')

grid_pattern = mb.pattern.Grid3DPattern(l=10, m=10, n=10)

lj_grid = grid_pattern.apply(compound=lj)

lj_box = mb.Compound()

for part in lj_grid:
    lj_box.add(part)

# saving out to the relevant files
lj_box.save(filename='lj_box.hoomdxml', overwrite=True, ref_distance=1)
lj_box.save(filename='lj_box.mol2', overwrite=True)

# initialize the simulation
hoomd.context.initialize("")
lattice = hoomd.deprecated.init.read_xml(filename='lj_box.hoomdxml', wrap_coordinates=True)

# distribute velocities to 
random.seed(1)
T_init = 0.1
for p in lattice.particles:
    p.velocity = (random.gauss(0, T_init), random.gauss(0, T_init), random.gauss(0, T_init))
    
# set inital neighbor list
nl = hoomd.md.nlist.cell(r_buff=0.6, check_period=1)

# set the integrator and groups
hoomd.md.integrate.mode_standard(dt=0.005)
all = hoomd.group.all()
nvt = hoomd.md.integrate.nvt(group=all, kT=0.2, tau=1.0)
hoomd.md.update.zero_momentum()

# set pair potential
lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)
lj.pair_coeff.set('C', 'C', epsilon=epsilon, sigma=sigma)

# optimize neighbor list r_buff
nl.tune(warmup=10000, set_max_check_period=False)

# set outputs
nvt_log = hoomd.analyze.log(filename="analyze_nvt.log",
                  quantities=['temperature', 'volume', 'kinetic_energy', 'potential_energy', 'pressure'],
                  period=100,
                  overwrite=True)
hoomd.dump.dcd("trajectory.dcd", period=1e3, group=all, overwrite=True, unwrap_full=True)

# run the simulation for 100000 timesteps
hoomd.run(1e5)

# set up npt run
nvt.disable()
nvt_log.disable()

npt = hoomd.md.integrate.npt(group=all, kT=0.5, tau=1.0, P=1.0, tauP=1.0)

hoomd.analyze.log(filename="analyze_npt.log",
                  quantities=['temperature', 'volume', 'kinetic_energy', 'potential_energy', 'pressure'],
                  period=100,
                  overwrite=True)

hoomd.run(1e5)

In [ ]:
data_nvt = np.genfromtxt(fname='analyze_nvt.log', skip_header=True);

# plot results
plt.figure();


plt.subplot(221);
plt.plot(data_nvt[:,0], data_nvt[:,2]);
plt.xlabel('time step');
plt.ylabel('volume');

plt.subplot(222);
plt.plot(data_nvt[:,0], data_nvt[:,1]);
plt.xlabel('time step');
plt.ylabel('temperature');

plt.subplot(223);
plt.plot(data_nvt[:,0], data_nvt[:,5]);
plt.xlabel('time step');
plt.ylabel('pressure');

plt.subplot(224);
plt.plot(data_nvt[:,0], data_nvt[:,3] + data_nvt[:,4]);
plt.xlabel('time step');
plt.ylabel('total_energy');

# adjust subplots so they do not overlap with each other
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=1.0, hspace=0.4,
                    wspace=0.35)

plt.show()

In [ ]:
data_npt = np.genfromtxt(fname='analyze_npt.log', skip_header=True);
plt.figure();


plt.subplot(221);
plt.plot(data_npt[:,0], data_npt[:,2]);
plt.xlabel('time step');
plt.ylabel('volume');

plt.subplot(222);
plt.plot(data_npt[:,0], data_npt[:,1]);
plt.xlabel('time step');
plt.ylabel('temperature');

plt.subplot(223);
plt.plot(data_npt[:,0], data_npt[:,5]);
plt.xlabel('time step');
plt.ylabel('pressure');

plt.subplot(224);
plt.plot(data_npt[:,0], data_npt[:,3] + data_npt[:,4]);
plt.xlabel('time step');
plt.ylabel('total_energy');

# adjust subplots so they do not overlap with each other
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=1.0, hspace=0.4,
                    wspace=0.35)

plt.show()

In [ ]:
# animate trajectory
traj = md.load(filename_or_filenames='./trajectory.dcd', top='./lj_box.mol2')
nv.show_mdtraj(traj)